<h1 style="background-color: rgb(236,138,1); height:40px; width: auto; border-radius: 5px; text-align: center">Introduction</h1>

<p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
This project will focus on historical sales data from 45 Walmart stores located in different regions.<br>
Each store is defined by a specific type, size and contains many departments.<br>
My goal is to use available data for be able to predict the sales for each department in each store.<br>
Special point is the presence of holiday markdown events included in the dataset.<br>
These markdowns are known to affect sales, it will make prediction which how departments are affected harder.
</p>

<h1 style="background-color: rgb(236,138,1); height:40px; width: auto; border-radius: 5px; text-align: center">Libraries and Tools for the project</h1>

<h2>Data Handling</h2>
<ul>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Pandas</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Pandas is a powerful data manipulation and analysis library.<br>
    It provides two primary data structures:<br>
    DataFrame: A 2D labeled data structure with columns of potentially different types.<br>
    Series: A 1D array-like object.
    </p>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>NumPy</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    NumPy is a library used for numerical computing.<br>
    It provides support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on.<br>
    </p>
</ul>

In [11]:
# Data Handling
import pandas as pd
import numpy as np

<h2>Data Visualization</h2>
<ul>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Plotly Express</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Plotly Express is a high-level interface for creating interactive plots easily.<br>
    It simplifies the process of creating visualizations like bar charts, scatter plots, line charts... with minimal code.
    </p>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Plotly Graph Objects</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Is a lower-level Plotly API for creating more customized and complex visualizations.<br>
    It allows for more detailed control over each component of the graph, such as layout and styling.
    </p>
    <li style="font-family: 'Monaco'; font-size: 15px; text-align: center; text-decoration: underline; list-style-type: none;"><b>Plotly Figure Factory</b></li>
    <p style="font-family: 'Menlo','Consolas'; font-size: 13px; text-align: center">
    Figure factory provides helper functions for creating complex plots like heatmaps, dendrograms, and tables that are otherwise more complex to build manually.
    </p>
</ul>

In [12]:
# Data Visualization
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [14]:

data = {
    'column1': [1, 2, 3, 4],
    'column2': ['a', 'b', 'c', 'd']
}

# Step 3: Create DataFrame
df = pd.DataFrame(data)

# Print the DataFrame to verify
print(df)

   column1 column2
0        1       a
1        2       b
2        3       c
3        4       d
